In [137]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [138]:
pd.set_option('display.max_row', 10)
df = pd.read_csv('../website_classification.csv')
df.tail(10)

,website_url,cleaned_website_text,Category
953,http://plasticwrevezcuvtt6aj2bl3xx5q4x4wm2ptlf...,plastic marketplacechoose plasticcloned cards ...,Financial Crime
954,http://ccpalymn3bi5uctrujnvrp5lf2nmyqf37oykanp...,ccpal ccs cvvs paypals ebay accounts and more ...,Financial Crime
955,http://imperiacfwnxzhkpyce2ifxwdf3zromko5wwrba...,imperial market the financial servicecard card...,Financial Crime
956,http://acteam2mf2ubo5hapf2dgsj4h7kjembfy4c6pml...,card the world verified sellersstore for credi...,Financial Crime
957,http://blackstgivygqcitkvmi6qwta3fpxhaakdjbhfe...,black store worldwide credit cards what we off...,Financial Crime
958,http://ccvendoqij2gteltjdd64usgdparw6kq2c2aqgm...,cc vendor qualityour prices usd cards eur card...,Financial Crime
959,https://ascarding.com/,the carding asylumcarding hacking vendors card...,Financial Crime
960,https://ascarding.com/forums/16/,cardable sites the carding asylumcardable site...,Financial Crime
961,https://ascarding.com/forums/19/,bank carding the carding asylumbank carding go...,Financial Crime
962,https://cardvilla.cc/carding-forum/?s=76185e0a...,carding forumcardingforumcarding carding carde...,Financial Crime


In [139]:
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


class ScrapTool:   
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        #Set up Selenium webdriver
        PATH = 'C:/Users/User01/Downloads/chromedriver_win32/chromedriver.exe'

        # Set the Chrome webdriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the Chrome webdriver and pass the Chrome options
        service = Service(PATH)
        driver = webdriver.Chrome(service = service, options = chrome_options)

        #Load website
        driver.get(website_url)
        
        # Check if the cookie consent button is present
        cookie_button = None
        try:
            cookie_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "cookie-consent-button")))
        except:
            pass

        # Handle cookies if the button is present
        if cookie_button:
            cookie_button.click()

        # Extract HTML content
        html_content = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html_content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                    self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [140]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+',' ',text)
    return text

In [141]:
def content_generation(website):
    try:
        scrapTool = ScrapTool()
        web = scrapTool.visit_url(website)
        text = cleaning_text(web['website_text'])
        return text
    except Exception as e:
        print('Error: ',website)
        print(e)
        return None

In [142]:
websites = [
    
]

for website in websites:
    try:
        content = content_generation(website)
        # content_list = [website,content,"Financial Crime"]
        # df.loc[len(df)] = content_list
    except Exception as e:
        print(website,": ",e)


In [143]:
df['Category'].value_counts()

Computers and Technology           302
Social Networking and Messaging    114
Business/Corporate                 106
E-Commerce                         101
News                                93
Law and Government                  83
Narcotics                           74
Adult                               43
Financial Crime                     31
Forums                              16
Name: Category, dtype: int64

In [144]:
from wordcloud import WordCloud

category_name = "Forums"
category_data = df[df["Category"]==category_name]['cleaned_website_text']
category_text = ' '.join(category_data)

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(category_text)

important_words = wordcloud.words_.keys()


In [146]:
words = ['hack','hacking','hacker','privacy policy','security']
words = words + list(important_words)
df.loc[df['Category'] == 'Financial Crime', 'cleaned_website_text'] = df.loc[df['Category'] == 'Financial Crime', 'cleaned_website_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in words]))

In [147]:
df[df['Category'] == 'Financial Crime']["cleaned_website_text"]

803    black bones cardingblack bones carding dumps c...
933    carding fraud wikipediacontents carding fraud ...
934    sell cc cvv clubcrd кардинг форум carding foru...
935    кардинг форум carding forumкардинг форум cardi...
936    кардинг форум carding forumкардинг форум cardi...
                             ...                        
958    cc vendor qualityour prices usd cards eur card...
959    the carding asylumcarding vendors carding manu...
960    cardable sites the carding asylumcardable site...
961    bank carding the carding asylumbank carding go...
962    carding forumcardingforumcarding carding carde...
Name: cleaned_website_text, Length: 31, dtype: object

In [133]:
df.to_csv("website_classification.csv",index=False)
# computers.to_csv("../Hierarchal model/Computer/computer_subcategory_classification.csv",index=False)